In [ ]:
%%capture
!pip install roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("kerem-kosif").project("competition-dataset")
dataset = project.version(6).download("coco")

In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import os
import json
import random
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import glob

from detectron2.structures import BoxMode
from detectron2 import model_zoo 
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog

In [ ]:
ann_path = "/content/competition-dataset-6/train/_annotations.coco.json"
with open(ann_path, "r") as jsonn:
    train_img_dicts = json.load(jsonn)

ann_path = "/content/competition-dataset-6/valid/_annotations.coco.json"
with open(ann_path, "r") as jsonn:
    val_img_dicts = json.load(jsonn)

In [ ]:
DatasetCatalog.clear()
MetadataCatalog.clear()

In [ ]:
from detectron2.data.datasets import register_coco_instances

for dataset_name in ["train", "valid"]:
    print(f"Registering {dataset_name} dataset!")
    ann_path = "/content/competition-dataset-6/" + dataset_name + "/_annotations.coco.json"
    file_path = "/content/competition-dataset-6/" + dataset_name
    
    register_coco_instances(dataset_name, {}, ann_path, file_path)


Registering train dataset!
Registering valid dataset!


In [ ]:
MetadataCatalog.clear()
MetadataCatalog.get("train").set(thing_classes=['cars-pedestrians', 'car', 'pedestrian'])
metadata = MetadataCatalog.get("train")

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg=get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")

cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[16, 32, 64, 128, 256]]
cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.5, 1.0, 2.0]]

cfg.INPUT.MIN_SIZE_TRAIN = 0
cfg.INPUT.MAX_SIZE_TRAIN = 99999
#cfg.INPUT.RANDOM_FLIP = "horizontal"

cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ("valid",)

cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2

cfg.SOLVER.BASE_LR = 0.00125
cfg.SOLVER.MAX_ITER = 700

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)

In [ ]:
trainer.train()

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.INPUT.MIN_SIZE_TEST = 0
cfg.INPUT.MAX_SIZE_TEST = 99999
cfg.DATASETS.TEST = ("valid",)
predictor = DefaultPredictor(cfg)

In [ ]:
file_path = "/content/competition-dataset-6/valid/"
image_num = 25
%matplotlib inline
for label in random.sample(val_img_dicts["images"], image_num):
    path = file_path + label["file_name"]
    print(path)
    plt.figure(figsize=(16,10))
    img = cv2.imread(path)
    outputs = predictor(img)
    visualizer = Visualizer(img[:, :, ::-1],
                          metadata=metadata,
                          scale=1)
    visualizer = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.imshow(visualizer.get_image()[:, :, ::-1])

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

evaluator = COCOEvaluator(dataset_name="valid", distributed=False,output_dir="./output")
val_loader = build_detection_test_loader(cfg, dataset_name="valid")
inference_on_dataset(model=trainer.model, data_loader=val_loader, evaluator=evaluator)